Doing some clusterization

In [1]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random
import os
import scipy.stats as ss
import scipy
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier, Pool, sum_models, to_classifier
from sklearn.model_selection import KFold

import implicit

In [2]:
df2 = pd.read_csv("../input/irkutsk/train_3lags_semibalanced.csv", low_memory=False)

In [ ]:
#df2 = pd.read_csv("../input/irkutsk/train_3lags_v4.csv", low_memory=False)

In [ ]:
#df2.drop(['Unnamed: 0'], axis=1, inplace=True)

In [122]:
df2 = df2.sample(frac=1).reset_index(drop=True)

In [4]:
df_test = df2[['service_title', 'order_count']]

In [5]:
df2.drop(['service',
'service_title',
'mfc',
'internal_status',
'external_status',
'order_type',
'department_id',
'custom_service_id',
'service_level',
'is_subdep',
'is_csid',
'proc_time',
'dayofweek',
'day_part',
'person',
'sole',
'legal',
'auto_ping_queue',
'win_count',
'month',
'week',
'year'], axis=1 , inplace=True)

In [6]:
df_train = df2.loc[df2['order_count'] > 1]

In [7]:
df_test = df_test.loc[df_test['order_count'] > 1]

In [8]:
df_train.reset_index(inplace=True)
df_test.reset_index(inplace=True)

In [9]:
df_train.drop(['index'], axis=1, inplace=True)
df_test.drop(['index'], axis=1, inplace=True)

In [ ]:
#mask = (df_train_mask['order_count'] == 5)
#z_valid = df_train_mask[mask]

#df_train_mask.loc[mask, 'service_4'] = np.nan


In [ ]:
df_test

In [ ]:
df_train.columns

In [10]:
categorical = ['service_1', 'service_title_1', 'mfc_1',
       'internal_status_1', 'external_status_1', 'order_type_1',
       'department_id_1', 'custom_service_id_1', 'service_level_1',
       'is_subdep_1', 'is_csid_1', 'dayofweek_1', 'day_part_1', 'month_1', 'week_1', 'year_1',
       'person_1', 'sole_1', 'legal_1', 'auto_ping_queue_1',
       'service_2', 'service_title_2', 'mfc_2', 'internal_status_2',
       'external_status_2', 'order_type_2', 'department_id_2',
       'custom_service_id_2', 'service_level_2', 'is_subdep_2', 'is_csid_2',
       'dayofweek_2', 'day_part_2', 'person_2', 'sole_2', 'month_2', 'week_2', 'year_2',
       'legal_2', 'auto_ping_queue_2','service_3',
       'service_title_3', 'mfc_3', 'internal_status_3', 'external_status_3',
       'order_type_3', 'department_id_3', 'custom_service_id_3',
       'service_level_3', 'is_subdep_3', 'is_csid_3', 'month_3', 'week_3', 'year_3',
       'dayofweek_3', 'day_part_3', 'person_3', 'sole_3', 'legal_3',
       'auto_ping_queue_3',
       'requester_type', 'gender']

In [11]:
cat = ['service_1', 'service_title_1', 'mfc_1',
       'internal_status_1', 'external_status_1',
       'department_id_1', 'custom_service_id_1', 'month_1', 'week_1', 'year_1',
       'is_subdep_1', 'is_csid_1', 'dayofweek_1',
       'service_2', 'service_title_2', 'mfc_2', 'internal_status_2',
       'external_status_2', 'department_id_2', 'month_2', 'week_2', 'year_2',
       'custom_service_id_2', 'is_subdep_2', 'is_csid_2',
       'dayofweek_2', 'service_3',
       'service_title_3', 'mfc_3', 'internal_status_3', 'external_status_3',
       'department_id_3', 'custom_service_id_3',
       'is_subdep_3', 'is_csid_3', 'month_3', 'week_3', 'year_3',
       'dayofweek_3', 
       'requester_type', 'gender']

In [12]:
X = df_train.drop(['requester'], axis=1)
y = df_test['service_title']

In [13]:
X.drop(['service_2',
'service_title_2',
'mfc_2',
'internal_status_2',
'external_status_2',
'order_type_2',
'department_id_2',
'custom_service_id_2',
'service_level_2',
'is_subdep_2',
'is_csid_2',
'proc_time_2',
'dayofweek_2',
'day_part_2',
'person_2',
'sole_2',
'legal_2',
'auto_ping_queue_2',
'win_count_2',
'month_2',
'week_2',
'year_2'], axis=1 , inplace=True)

X.drop(['service_3',
'service_title_3',
'mfc_3',
'internal_status_3',
'external_status_3',
'order_type_3',
'department_id_3',
'custom_service_id_3',
'service_level_3',
'is_subdep_3',
'is_csid_3',
'proc_time_3',
'dayofweek_3',
'day_part_3',
'person_3',
'sole_3',
'legal_3',
'auto_ping_queue_3',
'win_count_3',
'month_3',
'week_3',
'year_3'], axis=1 , inplace=True)

In [14]:
categorical = ['service_1', 'service_title_1', 'mfc_1',
       'internal_status_1', 'external_status_1', 'order_type_1',
       'department_id_1', 'custom_service_id_1', 'service_level_1',
       'is_subdep_1', 'is_csid_1', 'dayofweek_1', 'day_part_1', 'month_1', 'week_1', 'year_1',
       'person_1', 'sole_1', 'legal_1', 'auto_ping_queue_1',
       'requester_type', 'gender']

cat = ['service_1', 'service_title_1', 'mfc_1',
       'internal_status_1', 'external_status_1',
       'department_id_1', 'custom_service_id_1', 'month_1', 'week_1', 'year_1',
       'is_subdep_1', 'is_csid_1', 'dayofweek_1', 
       'requester_type', 'gender']

In [15]:
X[cat] = X[cat].astype('Int64')
X[cat] = X[cat].astype('object')

In [ ]:
#X[cat] = X[cat].astype('Int64')
#X[cat] = X[cat].astype('object')

In [16]:
X[categorical] = X[categorical].fillna('NA')

In [17]:
def reduce_mem_usage(df):
    """ 
    iterate through all the columns of a dataframe and 
    modify the data type to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print(('Memory usage of dataframe is {:.2f}' 
                     'MB').format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max <\
                  np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max <\
                   np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max <\
                   np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max <\
                   np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max <\
                   np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max <\
                   np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
    end_mem = df.memory_usage().sum() / 1024**2
    print(('Memory usage after optimization is: {:.2f}' 
                              'MB').format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) 
                                             / start_mem))
    
    return df

In [18]:
X = reduce_mem_usage(X)

Memory usage of dataframe is 252.21MB
Memory usage after optimization is: 75.66MB
Decreased by 70.0%


In [19]:
X = X[categorical]

In [20]:
X['person_1'] = X['person_1'].astype('int32')
X['sole_1'] = X['sole_1'].astype('int32')
X['legal_1'] = X['legal_1'].astype('int32')
X['auto_ping_queue_1'] = X['auto_ping_queue_1'].astype('int32')

In [21]:
from sklearn import preprocessing
labeling = []
for col in X.columns:
    d = pd.DataFrame()
    d[col] = X[col].unique()
    le = preprocessing.LabelEncoder()
    le.fit(d[col])
    d[col+'_l'] = le.transform(d[col])
    d = d.sort_values(by=[col+'_l']).reset_index(drop=True)
    labeling.append(d)

In [22]:
d = pd.DataFrame()
d['service_title'] = y.unique()
le = preprocessing.LabelEncoder()
le.fit(d['service_title'])
d['service_title'+'_l'] = le.transform(d['service_title'])
d = d.sort_values(by=['service_title'+'_l']).reset_index(drop=True)
labeling_y = d

In [ ]:
labeling[0]

In [23]:
i = 0
for col in X.columns:
    X[col] = X[col].map(labeling[i].set_index(col).to_dict()[col+'_l'])
    i += 1

In [ ]:
X

In [24]:
y = y.map(labeling_y.set_index('service_title').to_dict()['service_title'+'_l'])

In [ ]:
for col in X.columns:
    print(col, len(X[col].unique()))

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

BATCH_SIZE = 64 * strategy.num_replicas_in_sync
LEARNING_RATE = 3e-5 * strategy.num_replicas_in_sync 
EPOCHS = 5

In [ ]:
X.columns

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [ ]:
y_test.value_counts(normalize=True)

In [ ]:
len(X['service_1'].unique())

In [ ]:
def generate_dataset(idxT, idxV):    
    
    trn_weights = weights[idxT, ]
    val_weights = weights[idxV, ]
    #n samples
    
    trn_input_ids = np.array(X.index)[idxT,]
    
    # Trainset
    trn_input_service_1 = np.array(X['service_1'])[idxT,]
    trn_input_service_title_1 = np.array(X['service_title_1'])[idxT,]
    trn_input_mfc_1 = np.array(X['mfc_1'])[idxT,]
    trn_input_internal_status_1 = np.array(X['internal_status_1'])[idxT,]
    trn_input_external_status_1 = np.array(X['external_status_1'])[idxT,]
    trn_input_order_type_1 = np.array(X['order_type_1'])[idxT,]
    trn_input_department_id_1 = np.array(X['department_id_1'])[idxT,]
    trn_input_custom_service_id_1 = np.array(X['custom_service_id_1'])[idxT,]
    trn_input_service_level_1 = np.array(X['service_level_1'])[idxT,]
    trn_input_is_subdep_1 = np.array(X['is_subdep_1'])[idxT,]
    trn_input_is_csid_1 = np.array(X['is_csid_1'])[idxT,]
    trn_input_dayofweek_1 = np.array(X['dayofweek_1'])[idxT,]
    trn_input_day_part_1 = np.array(X['day_part_1'])[idxT,]
    trn_input_month_1 = np.array(X['month_1'])[idxT,]
    trn_input_week_1 = np.array(X['week_1'])[idxT,]
    trn_input_year_1 = np.array(X['year_1'])[idxT,]
    trn_input_person_1 = np.array(X['person_1'])[idxT,]
    trn_input_sole_1 = np.array(X['sole_1'])[idxT,]
    trn_input_legal_1 = np.array(X['legal_1'])[idxT,]
    trn_input_auto_ping_queue_1 = np.array(X['auto_ping_queue_1'])[idxT,]
    trn_input_requester_type = np.array(X['requester_type'])[idxT,]
    trn_input_gender = np.array(X['gender'])[idxT,]
    
    
    trn_service_title = np.array(pd.get_dummies(y))[idxT,].astype('int32')
    
    # Validation set
    val_input_service_1 = np.array(X['service_1'])[idxV,]
    val_input_service_title_1 = np.array(X['service_title_1'])[idxV,]
    val_input_mfc_1 = np.array(X['mfc_1'])[idxV,]
    val_input_internal_status_1 = np.array(X['internal_status_1'])[idxV,]
    val_input_external_status_1 = np.array(X['external_status_1'])[idxV,]
    val_input_order_type_1 = np.array(X['order_type_1'])[idxV,]
    val_input_department_id_1 = np.array(X['department_id_1'])[idxV,]
    val_input_custom_service_id_1 = np.array(X['custom_service_id_1'])[idxV,]
    val_input_service_level_1 = np.array(X['service_level_1'])[idxV,]
    val_input_is_subdep_1 = np.array(X['is_subdep_1'])[idxV,]
    val_input_is_csid_1 = np.array(X['is_csid_1'])[idxV,]
    val_input_dayofweek_1 = np.array(X['dayofweek_1'])[idxV,]
    val_input_day_part_1 = np.array(X['day_part_1'])[idxV,]
    val_input_month_1 = np.array(X['month_1'])[idxV,]
    val_input_week_1 = np.array(X['week_1'])[idxV,]
    val_input_year_1 = np.array(X['year_1'])[idxV,]
    val_input_person_1 = np.array(X['person_1'])[idxV,]
    val_input_sole_1 = np.array(X['sole_1'])[idxV,]
    val_input_legal_1 = np.array(X['legal_1'])[idxV,]
    val_input_auto_ping_queue_1 = np.array(X['auto_ping_queue_1'])[idxV,]
    val_input_requester_type = np.array(X['requester_type'])[idxV,]
    val_input_gender = np.array(X['gender'])[idxV,]
    
    
    val_service_title = np.array(pd.get_dummies(y))[idxV,].astype('int32')
    
    # Generating tf.data object
    train_dataset = (
        tf.data.Dataset
        .from_tensor_slices(({'service_1':trn_input_service_1, 
                              'service_title_1': trn_input_service_title_1,
                              'mfc_1': trn_input_mfc_1,
                              'internal_status_1': trn_input_internal_status_1,
                              'external_status_1': trn_input_external_status_1,
                              'order_type_1': trn_input_order_type_1,
                              'department_id_1': trn_input_department_id_1,
                              'custom_service_id_1': trn_input_custom_service_id_1,
                              'service_level_1': trn_input_service_level_1,
                              'is_subdep_1': trn_input_is_subdep_1,
                              'is_csid_1': trn_input_is_csid_1,
                              'dayofweek_1': trn_input_dayofweek_1,
                              'day_part_1': trn_input_day_part_1,
                              'month_1': trn_input_month_1,
                              'week_1': trn_input_week_1,
                              'year_1': trn_input_year_1,
                              'person_1': trn_input_person_1,
                              'sole_1': trn_input_sole_1,
                              'legal_1': trn_input_legal_1,
                              'auto_ping_queue_1': trn_input_auto_ping_queue_1,
                              'requester_type': trn_input_requester_type,
                              'gender': trn_input_gender}, 
                             {'service_title': trn_service_title}, trn_weights))
        .shuffle(2048)
        .batch(BATCH_SIZE)
        .prefetch(AUTO)
    )
    
    valid_dataset = (
        tf.data.Dataset
        .from_tensor_slices(({'service_1':val_input_service_1, 
                              'service_title_1': val_input_service_title_1,
                              'mfc_1': val_input_mfc_1,
                              'internal_status_1': val_input_internal_status_1,
                              'external_status_1': val_input_external_status_1,
                              'order_type_1': val_input_order_type_1,
                              'department_id_1': val_input_department_id_1,
                              'custom_service_id_1': val_input_custom_service_id_1,
                              'service_level_1': val_input_service_level_1,
                              'is_subdep_1': val_input_is_subdep_1,
                              'is_csid_1': val_input_is_csid_1,
                              'dayofweek_1': val_input_dayofweek_1,
                              'day_part_1': val_input_day_part_1,
                              'month_1': val_input_month_1,
                              'week_1': val_input_week_1,
                              'year_1': val_input_year_1,
                              'person_1': val_input_person_1,
                              'sole_1': val_input_sole_1,
                              'legal_1': val_input_legal_1,
                              'auto_ping_queue_1': val_input_auto_ping_queue_1,
                              'requester_type': val_input_requester_type,
                              'gender': val_input_gender}, 
                             {'service_title': val_service_title}, val_weights))
        .batch(BATCH_SIZE)
        .cache()
        .prefetch(AUTO)
    )
    
    return trn_input_ids.shape[0]//BATCH_SIZE, train_dataset, valid_dataset

In [ ]:
def scheduler(epoch):
    return LEARNING_RATE * 0.2**epoch

In [ ]:
np.array(pd.get_dummies(y))[[0, 1],].astype('int32')

In [ ]:
def build_model():
    service_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='service_1')
    service_title_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='service_title_1')
    mfc_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='mfc_1')
    internal_status_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='internal_status_1')
    external_status_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='external_status_1')
    order_type_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='order_type_1')
    department_id_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='department_id_1')
    custom_service_id_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='custom_service_id_1')
    service_level_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='service_level_1')
    is_subdep_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='is_subdep_1')
    is_csid_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='is_csid_1')
    dayofweek_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='dayofweek_1')
    day_part_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='day_part_1')
    month_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='month_1')
    week_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='week_1')
    year_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='year_1')
    person_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='person_1')
    sole_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='sole_1')
    legal_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='legal_1')
    auto_ping_queue_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='auto_ping_queue_1')
    requester_type = tf.keras.layers.Input((1,), dtype=tf.int32, name='requester_type')
    gender = tf.keras.layers.Input((1,), dtype=tf.int32, name='gender')
    
    service_1_embedding = tf.keras.layers.Embedding(len(X['service_1'].unique()), 11, input_length=1, name='service_1_embedding')(service_1)
    service_title_1_embedding = tf.keras.layers.Embedding(len(X['service_title_1'].unique()), 50, input_length=1, name='service_title_1_embedding')(service_title_1)
    mfc_1_embedding = tf.keras.layers.Embedding(len(X['mfc_1'].unique()), 50, input_length=1, name='mfc_1_embedding')(mfc_1)
    internal_status_1_embedding = tf.keras.layers.Embedding(len(X['internal_status_1'].unique()), 5, input_length=1, name='internal_status_1_embedding')(internal_status_1)
    external_status_1_embedding = tf.keras.layers.Embedding(len(X['external_status_1'].unique()), 14, input_length=1, name='external_status_1_embedding')(external_status_1)
    order_type_1_embedding = tf.keras.layers.Embedding(len(X['order_type_1'].unique()), 2, input_length=1, name='order_type_1_embedding')(order_type_1)
    department_id_1_embedding = tf.keras.layers.Embedding(len(X['department_id_1'].unique()), 46, input_length=1, name='department_id_1_embedding')(department_id_1)
    custom_service_id_1_embedding = tf.keras.layers.Embedding(len(X['custom_service_id_1'].unique()), 26, input_length=1, name='custom_service_id_1_embedding')(custom_service_id_1)
    service_level_1_embedding = tf.keras.layers.Embedding(len(X['service_level_1'].unique()), 3, input_length=1, name='service_level_1_embedding')(service_level_1)
    is_subdep_1_embedding = tf.keras.layers.Embedding(len(X['is_subdep_1'].unique()), 1, input_length=1, name='is_subdep_1_embedding')(is_subdep_1)
    is_csid_1_embedding = tf.keras.layers.Embedding(len(X['is_csid_1'].unique()), 1, input_length=1, name='is_csid_1_embedding')(is_csid_1)
    dayofweek_1_embedding = tf.keras.layers.Embedding(len(X['dayofweek_1'].unique()), 4, input_length=1, name='dayofweek_1_embedding')(dayofweek_1)
    day_part_1_embedding = tf.keras.layers.Embedding(len(X['day_part_1'].unique()), 2, input_length=1, name='day_part_1_embedding')(day_part_1)
    month_1_embedding = tf.keras.layers.Embedding(len(X['month_1'].unique()), 6, input_length=1, name='month_1_embedding')(month_1)
    week_1_embedding = tf.keras.layers.Embedding(len(X['week_1'].unique()), 26, input_length=1, name='week_1_embedding')(week_1)
    year_1_embedding = tf.keras.layers.Embedding(len(X['year_1'].unique()), 1, input_length=1, name='year_1_embedding')(year_1)
    person_1_embedding = tf.keras.layers.Embedding(len(X['person_1'].unique()), 1, input_length=1, name='person_1_embedding')(person_1)
    sole_1_embedding = tf.keras.layers.Embedding(len(X['sole_1'].unique()), 1, input_length=1, name='sole_1_embedding')(sole_1)
    legal_1_embedding = tf.keras.layers.Embedding(len(X['legal_1'].unique()), 1, input_length=1, name='legal_1_embedding')(legal_1)
    auto_ping_queue_1_embedding = tf.keras.layers.Embedding(len(X['auto_ping_queue_1'].unique()), 1, input_length=1, name='auto_ping_queue_1_embedding')(auto_ping_queue_1)
    requester_type_embedding = tf.keras.layers.Embedding(len(X['requester_type'].unique()), 2, input_length=1, name='requester_type_embedding')(requester_type)
    gender_embedding = tf.keras.layers.Embedding(len(X['gender'].unique()), 1, input_length=1, name='gender_embedding')(gender)

    concatenated = tf.keras.layers.Concatenate()([service_1_embedding, 
                                                  service_title_1_embedding,
                                                 mfc_1_embedding,
                                                 internal_status_1_embedding,
                                                 external_status_1_embedding,
                                                 order_type_1_embedding,
                                                 department_id_1_embedding,
                                                 custom_service_id_1_embedding,
                                                 service_level_1_embedding,
                                                 is_subdep_1_embedding,
                                                 is_csid_1_embedding,
                                                 dayofweek_1_embedding,
                                                 day_part_1_embedding,
                                                 month_1_embedding,
                                                 year_1_embedding,
                                                 person_1_embedding,
                                                 sole_1_embedding,
                                                 legal_1_embedding,
                                                 auto_ping_queue_1_embedding,
                                                 requester_type_embedding,
                                                 gender_embedding])
    #out = tf.keras.layers.Flatten()(concatenated)
    
    #out = tf.keras.layers.Dense(512, activation='relu')(out)
    #out = tf.keras.layers.Dense(256, activation='relu')(out)
    #out = tf.keras.layers.Dense(256, activation='relu')(out)
    #out = tf.keras.layers.Conv1D(128, 2, padding='same')(concatenated)
    #out = tf.keras.layers.LeakyReLU()(out)
    #out = tf.keras.layers.Conv1D(64, 2, padding='same')(out)
    #out = tf.keras.layers.Flatten()(out)
    conv_0 = tf.keras.layers.Conv1D(128, 3, padding='same', activation='relu')(concatenated)
    conv_1 = tf.keras.layers.Conv1D(128, 2, padding='same', activation='relu')(concatenated)
    conv_2 = tf.keras.layers.Conv1D(128, 6, padding='same', activation='relu')(concatenated)
    conv_0 = tf.keras.layers.Conv1D(64, 3, padding='same', activation='relu')(conv_0)
    conv_1 = tf.keras.layers.Conv1D(64, 2, padding='same', activation='relu')(conv_1)
    conv_2 = tf.keras.layers.Conv1D(64, 6, padding='same', activation='relu')(conv_2)
    concatenated_tensor = tf.keras.layers.Concatenate(axis=1)([conv_0, conv_1, conv_2])
    out = tf.keras.layers.Flatten()(concatenated_tensor)
    
    out = tf.keras.layers.Dense(len(y.unique()), activation='softmax', name='service_title')(out)

    model = tf.keras.models.Model(inputs=[service_1, 
                                          service_title_1, 
                                          mfc_1,
                                         internal_status_1,
                                         external_status_1,
                                         order_type_1,
                                         department_id_1,
                                         custom_service_id_1,
                                         service_level_1,
                                         is_subdep_1,
                                         is_csid_1,
                                         dayofweek_1,
                                         day_part_1,
                                         month_1,
                                         week_1,
                                         year_1,
                                         person_1,
                                         sole_1,
                                         legal_1,
                                         auto_ping_queue_1,
                                         requester_type,
                                         gender], 
                                  outputs=out)

    optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(loss = 'categorical_crossentropy', optimizer=optimizer, metrics=["accuracy"])

    return model

In [ ]:
inputs = [np.array(X_test['service_1']),
np.array(X_test['service_title_1']),
np.array(X_test['mfc_1']),
np.array(X_test['internal_status_1']),
np.array(X_test['external_status_1']),
np.array(X_test['order_type_1']),
np.array(X_test['department_id_1']),
np.array(X_test['custom_service_id_1']),
np.array(X_test['service_level_1']),
np.array(X_test['is_subdep_1']),
np.array(X_test['is_csid_1']),
np.array(X_test['dayofweek_1']),
np.array(X_test['day_part_1']),
np.array(X_test['month_1']),
np.array(X_test['week_1']),
np.array(X_test['year_1']),
np.array(X_test['person_1']),
np.array(X_test['sole_1']),
np.array(X_test['legal_1']),
np.array(X_test['auto_ping_queue_1']),
np.array(X_test['requester_type']),
np.array(X_test['gender'])]

In [ ]:
kfold_preds = np.zeros((X_test.shape[0], labeling_y.shape[0]))
n_splits = 5
for i in range(5):
    print('#'*25)
    print('### MODEL %i'%(i+1))
    print('#'*25)
    
    K.clear_session()
    model = build_model()
    model.load_weights('../input/categorical-embedding-v2/v6-learnedemb-%i.h5'%i)

    print('Predicting Test...')
    preds = model.predict(inputs,verbose=True)
    kfold_preds += preds/n_splits

In [ ]:
y_pred = []
for k in range(y_test.shape[0]):
    a = np.argmax(preds[k,])
    y_pred.append(a)

In [ ]:
accuracy_score(y_pred, y_test)

In [25]:
df = pd.read_csv("../input/feature-creating/train_3lags_v1.csv", low_memory=False)

In [26]:
sab = pd.read_csv("../input/irkutsk/sample_submission.csv")

sab_pred = sab[['requester']]

sab_pred = pd.merge(sab_pred, df, how='left', on='requester')

sab_pred.drop(['service_3',
'service_title_3',
'mfc_3',
'internal_status_3',
'external_status_3',
'order_type_3',
'department_id_3',
'custom_service_id_3',
'service_level_3',
'is_subdep_3',
'is_csid_3',
'proc_time_3',
'dayofweek_3',
'day_part_3',
'person_3',
'sole_3',
'legal_3',
'auto_ping_queue_3',
'win_count_3', 'year_3', 'month_3', 'week_3'], axis=1 , inplace=True)

sab_pred.drop(['requester'], axis=1, inplace=True)

In [ ]:
df2.head()

In [ ]:
sab_pred.head()

In [ ]:
df2['age'].value_counts(normalize=True)

In [38]:
sab_pred['gender'].value_counts(normalize=True)

1    0.538565
0    0.461435
Name: gender, dtype: float64

In [ ]:
sab_pred.columns

In [27]:
sab_pred.drop(['service_2',
'service_title_2',
'mfc_2',
'internal_status_2',
'external_status_2',
'order_type_2',
'department_id_2',
'custom_service_id_2',
'service_level_2',
'is_subdep_2',
'is_csid_2',
'proc_time_2',
'dayofweek_2',
'day_part_2',
'person_2',
'sole_2',
'legal_2',
'auto_ping_queue_2',
'win_count_2',
'month_2',
'week_2',
'year_2'], axis=1 , inplace=True)

sab_pred.drop(['service_1',
'service_title_1',
'mfc_1',
'internal_status_1',
'external_status_1',
'order_type_1',
'department_id_1',
'custom_service_id_1',
'service_level_1',
'is_subdep_1',
'is_csid_1',
'proc_time_1',
'dayofweek_1',
'day_part_1',
'person_1',
'sole_1',
'legal_1',
'auto_ping_queue_1',
'win_count_1',
'month_1',
'week_1',
'year_1'], axis=1 , inplace=True)

In [ ]:
sab_pred.columns

In [ ]:
X.columns

In [28]:
sab_pred.drop(['proc_time', 'win_count', 'age', 'order_count'], axis=1, inplace=True)

In [29]:
new_cols=X.columns
sab_pred.rename(columns=dict(zip(sab_pred.columns, new_cols)),inplace=True)
sab_pred[cat] = sab_pred[cat].astype('Int64')
sab_pred[cat] = sab_pred[cat].astype('object')
#sab_pred[categorical] = sab_pred[categorical].fillna('NA')
#sab_pred = reduce_mem_usage(sab_pred)

In [30]:
sab_pred['person_1'] = sab_pred['person_1'].map({True: 1, False:0})
sab_pred['sole_1'] = sab_pred['sole_1'].map({True: 1, False:0})
sab_pred['legal_1'] = sab_pred['legal_1'].map({True: 1, False:0})
sab_pred['auto_ping_queue_1'] = sab_pred['auto_ping_queue_1'].map({True: 1, False:0})

In [31]:
i = 0
for col in sab_pred.columns:
    sab_pred[col] = sab_pred[col].map(labeling[i].set_index(col).to_dict()[col+'_l'])
    i += 1

In [32]:
sab_pred.fillna(0, inplace=True)

In [33]:
toint = ['service_1', 'service_title_1', 'mfc_1', 'order_type_1', 'person_1', 'sole_1', 'auto_ping_queue_1', 'day_part_1',
       'internal_status_1', 'external_status_1', 'service_level_1', 'legal_1',
       'department_id_1', 'custom_service_id_1', 'month_1', 'week_1', 'year_1',
       'is_subdep_1', 'is_csid_1', 'dayofweek_1', 
       'requester_type', 'gender']

In [34]:
sab_pred[toint] = sab_pred[toint].astype('int64')

In [35]:
sab_pred['order_type_1'] = sab_pred['order_type_1'].astype('category')
sab_pred['service_level_1'] = sab_pred['service_level_1'].astype('category')
sab_pred['day_part_1'] = sab_pred['day_part_1'].astype('category')

In [ ]:
inputs_t = [np.array(sab_pred['service_1']),
np.array(sab_pred['service_title_1']),
np.array(sab_pred['mfc_1']),
np.array(sab_pred['internal_status_1']),
np.array(sab_pred['external_status_1']),
np.array(sab_pred['order_type_1']),
np.array(sab_pred['department_id_1']),
np.array(sab_pred['custom_service_id_1']),
np.array(sab_pred['service_level_1']),
np.array(sab_pred['is_subdep_1']),
np.array(sab_pred['is_csid_1']),
np.array(sab_pred['dayofweek_1']),
np.array(sab_pred['day_part_1']),
np.array(sab_pred['month_1']),
np.array(sab_pred['week_1']),
np.array(sab_pred['year_1']),
np.array(sab_pred['person_1']),
np.array(sab_pred['sole_1']),
np.array(sab_pred['legal_1']),
np.array(sab_pred['auto_ping_queue_1']),
np.array(sab_pred['requester_type']),
np.array(sab_pred['gender'])]

In [ ]:
kfold_preds = np.zeros((sab_pred.shape[0], labeling_y.shape[0]))
n_splits = 5
for i in range(5):
    print('#'*25)
    print('### MODEL %i'%(i+1))
    print('#'*25)
    
    K.clear_session()
    model = build_model()
    model.load_weights('../input/categorical-embedding-v2/v6-learnedemb-%i.h5'%i)

    print('Predicting Test...')
    preds = model.predict(inputs_t,verbose=True)
    kfold_preds += preds/n_splits

In [ ]:
y_pred = []
for k in range(sab_pred.shape[0]):
    a = np.argmax(kfold_preds[k,])
    y_pred.append(a)

In [ ]:
sab['service_title'] = y_pred

In [ ]:
sab['service_title'] = sab['service_title'].map(labeling_y.set_index('service_title_l').to_dict()['service_title'])

In [ ]:
sab['service_title'].value_counts(normalize=True)[:30]

In [ ]:
sab.to_csv('nn_sub_5fold_6.csv', index=False)

In [55]:
from sklearn.preprocessing import StandardScaler
from kmodes.kprototypes import KPrototypes

In [42]:
sab_pred2 = sab[['requester']]

sab_pred2 = pd.merge(sab_pred2, df, how='left', on='requester')

In [44]:
sab_pred['age'] = sab_pred2['age']

In [46]:
sab_pred.fillna(0, inplace=True)

In [49]:
sab_cluster = sab_pred[['service_title_1', 'external_status_1', 'order_type_1', 'department_id_1', 'custom_service_id_1', 'service_level_1', 'requester_type', 'gender', 'age']]

In [56]:
scaler = StandardScaler()
scaler.fit(sab_cluster[['age']])
sab_cluster[['age']] = scaler.transform(sab_cluster[['age']])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [57]:
sab_cluster

,service_title_1,external_status_1,order_type_1,department_id_1,custom_service_id_1,service_level_1,requester_type,gender,age
0,993,4,0,16,18,4,0,1,-0.267723
1,48,4,0,16,1,4,0,0,-0.527816
2,110,21,3,8,28,0,0,0,1.136780
3,742,4,3,15,0,0,0,0,-0.423779
4,493,21,3,72,25,0,0,1,-0.111667
...,...,...,...,...,...,...,...,...,...
44439,168,1,3,72,25,0,0,0,-0.371760
44440,82,1,3,8,28,0,0,1,0.720631
44441,886,4,3,15,47,0,0,1,-0.267723
44442,404,4,3,15,43,0,0,1,-0.423779


In [58]:
sab_cluster_array = sab_cluster.values

In [61]:
sab_cluster_array

array([[993, 4, 0, ..., 0, 1, -0.2677228734177517],
       [48, 4, 0, ..., 0, 0, -0.5278159434387918],
       [110, 21, 3, ..., 0, 0, 1.136779704695865],
       ...,
       [886, 4, 3, ..., 0, 1, -0.2677228734177517],
       [404, 4, 3, ..., 0, 1, -0.42377871543037576],
       [404, 4, 3, ..., 0, 1, -0.3717601014261677]], dtype=object)

In [60]:
sab_cluster_array[:, 8] = sab_cluster_array[:, 8].astype(float)

In [64]:
#kproto = KPrototypes(n_clusters=45, verbose=2, max_iter=20)
#kproto.fit_predict(sab_cluster_array, categorical=[0, 1, 2, 3, 4, 5, 6, 7])

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/20, moves: 12562, ncost: 32718.842281616096
Run: 1, iteration: 2/20, moves: 4954, ncost: 32235.067498520293
Run: 1, iteration: 3/20, moves: 1840, ncost: 32017.470443981325
Run: 1, iteration: 4/20, moves: 1167, ncost: 31793.122260060853
Run: 1, iteration: 5/20, moves: 512, ncost: 31783.948920360726
Run: 1, iteration: 6/20, moves: 244, ncost: 31772.3504277282
Run: 1, iteration: 7/20, moves: 444, ncost: 31736.226663288504
Run: 1, iteration: 8/20, moves: 269, ncost: 31730.737454344984
Run: 1, iteration: 9/20, moves: 167, ncost: 31729.31551994296
Run: 1, iteration: 10/20, moves: 37, ncost: 31729.015693232785
Run: 1, iteration: 11/20, moves: 1, ncost: 31729.013094277874
Run: 1, iteration: 12/20, moves: 0, ncost: 31729.013094277874
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 2, iteration: 1/20, moves: 15057, ncost: 33301.578243911084
Run: 2, iteration: 

Run: 8, iteration: 20/20, moves: 252, ncost: 32992.46567167683
Run: 8, iteration: 21/20, moves: 104, ncost: 32987.91828138221
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 9, iteration: 1/20, moves: 15038, ncost: 34356.23610544287
Run: 9, iteration: 2/20, moves: 6849, ncost: 33265.42655270793
Run: 9, iteration: 3/20, moves: 3534, ncost: 32697.54574522374
Run: 9, iteration: 4/20, moves: 3499, ncost: 32218.43384492273
Run: 9, iteration: 5/20, moves: 1587, ncost: 32164.070740242172
Run: 9, iteration: 6/20, moves: 1046, ncost: 32136.14516693182
Run: 9, iteration: 7/20, moves: 473, ncost: 32121.764576062564
Run: 9, iteration: 8/20, moves: 400, ncost: 32111.506237165086
Run: 9, iteration: 9/20, moves: 314, ncost: 32102.928342377018
Run: 9, iteration: 10/20, moves: 246, ncost: 32096.897881355508
Run: 9, iteration: 11/20, moves: 120, ncost: 32095.70967613422
Run: 9, iteration: 12/20, moves: 75, ncost: 32093.680733529087
Run: 9, iteration: 13/20, moves: 26

array([34, 34, 33, ..., 24, 24, 24], dtype=uint16)

In [87]:
import joblib
#filename = 'kproto.sav'
#joblib.dump(kproto, filename)
#loaded_model = joblib.load(filename)

['kproto.sav']

In [196]:
filename = "../input/kproto/kproto.sav"
kproto = joblib.load(filename)

In [66]:
sab_pred['cluster'] = kproto.predict(sab_cluster_array, categorical=[0, 1, 2, 3, 4, 5, 6, 7])

In [73]:
X['age'] = df2['age']

In [74]:
X_cluster = X[['service_title_1', 'external_status_1', 'order_type_1', 'department_id_1', 'custom_service_id_1', 'service_level_1', 'requester_type', 'gender', 'age']]

In [76]:
X_cluster[['age']] = scaler.transform(X_cluster[['age']])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [78]:
X_cluster_array = X_cluster.values

In [79]:
X_cluster_array[:, 8] = X_cluster_array[:, 8].astype(float)

In [80]:
X['cluster'] = kproto.predict(X_cluster_array, categorical=[0, 1, 2, 3, 4, 5, 6, 7])

In [99]:
x_p_new = dict(sab_pred['cluster'].value_counts(normalize=True))

In [104]:
x = next(iter(x_p_new))

In [102]:
x_old = dict(X['cluster'].value_counts(normalize=False))

In [105]:
max_n = x_old[x]

In [107]:
max_p_new = x_p_new[x]

In [119]:
def get_cluster_size(xp_new, max_n, max_p_new):
    n_new = round(max_n/max_p_new)
    d_x_new = {}
    for i in list(xp_new.keys()):
        d_x_new[i] = int(round(n_new*xp_new[i]))
    return d_x_new    

In [120]:
x_new = get_cluster_size(x_p_new, max_n, max_p_new)

In [126]:
X['service_title'] = y

In [134]:
X.loc[X['cluster'] == 1].sample(n=1000).reset_index(drop=True)

,service_1,service_title_1,mfc_1,internal_status_1,external_status_1,order_type_1,department_id_1,custom_service_id_1,service_level_1,is_subdep_1,...,year_1,person_1,sole_1,legal_1,auto_ping_queue_1,requester_type,gender,age,cluster,service_title
0,13,946,140,7,21,3,72,0,0,0,...,0,1,0,0,1,0,1,41.0,1,126
1,0,640,1,7,21,3,34,11,0,1,...,0,0,0,0,1,0,1,34.0,1,10
2,0,896,85,7,21,1,69,1,3,0,...,0,0,0,0,1,0,1,34.0,1,136
3,0,432,139,7,21,3,34,10,0,1,...,0,0,0,0,1,0,1,33.0,1,90
4,13,946,49,7,4,0,72,0,0,0,...,0,1,0,0,1,0,1,32.0,1,126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,13,946,26,7,21,3,72,0,0,0,...,0,1,0,0,1,0,1,36.0,1,133
996,14,736,129,7,21,3,72,0,0,0,...,0,1,0,0,1,0,1,40.0,1,126
997,14,736,129,7,21,3,72,0,0,0,...,0,1,0,0,1,0,1,42.0,1,126
998,0,860,85,7,21,3,34,48,0,1,...,1,0,0,0,1,0,1,33.0,1,138


In [139]:
X_clustered = pd.DataFrame(columns = X.columns)

In [179]:
X_clustered2 = pd.DataFrame(columns = X.columns)
chunks = []
X2 = X.reset_index()
for i in list(x_new.keys()):
    chunk = X2.loc[X2['cluster'] == i][:x_new[i]]
    chunks.append(chunk)
X_clustered2 = pd.concat(chunks, ignore_index=True)

In [191]:
X_clustered2['service_title'] = X_clustered2['service_title'].map(labeling_y.set_index('service_title_l').to_dict()['service_title'])

In [183]:
i = 0
for col in categorical:
    X_clustered2[col] =  X_clustered2[col].map(labeling[i].set_index(col + '_l').to_dict()[col])
    i += 1

In [186]:
X_clustered2 = X_clustered2.sample(frac=1).reset_index(drop=True)

In [194]:
X_clustered2.to_csv('df_train_clustered.csv', index=False)